## Evaluating and visualizing gene ontology (GO) terms associated bacterial peptides in degradation experiment

#### Beginning with: stripped bacterial peptides and GO term abundances in spreadsheets

#### Goal: heatmaps of bacterial GO terms over the 4 timepoints

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from functools import reduce

## First, combining all bacterial peptides from all timepoints and treatments (trypsin digested and naturally digested) to get a hierarchy of all possible GO terms

In [11]:
# We want the stripped peptides of just bacterial LCA hits in UniPept.
# Navigate to where these LCA output are in csvs in git repo:

!ls /home/millieginty/Documents/git-repos/rot-mayer/analyses/unipept/lca/cleaned/

T0-322-trypsin-lca-DB.csv   T12-325-trypsin-lca-DN.csv
T0-322-trypsin-lca-DN.csv   T2-323-trypsin-lca-DN.csv
T12-325-trypsin-lca-DB.csv  T5-324-trypsin-lca-DN.csv


### 1. T0 322 trypsin PeaksDB (DB)

In [12]:
# read the data into pandas dataframes
T0_322_dig_DB = pd.read_csv("/home/millieginty/Documents/git-repos/rot-mayer/analyses/unipept/lca/cleaned/T0-322-trypsin-lca-DB.csv", index_col=0)

T0_322_dig_DB.head()

,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,...,tribe,subtribe,genus,subgenus,species,group,species.1,subgroup,species.2,subspecies
peptide,,,,,,,,,,,,,,,,,,,,,
LPQVEGTGGDVQPSQDLVR,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VLGQNEAVDAVSNALR,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALDLLDEAASSLR,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VTDAELAEVLAR,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STEFDNLLLVGPLAGK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Take just the bacterial peptides

# keep only the rows where kingdom = Bacteria

T0_322_dig_DB_Bac1 = T0_322_dig_DB.loc[(T0_322_dig_DB['kingdom'] == 'Bacteria')]

T0_322_dig_DB_Bac2 = T0_322_dig_DB.loc[(T0_322_dig_DB['subkingdom'] == 'Bacteria')]

print('# of kingdom = ', len(T0_322_dig_DB_Bac1))

print('# of subkingdom = ', len(T0_322_dig_DB_Bac2))

frames = [T0_322_dig_DB_Bac1, T0_322_dig_DB_Bac2]
  
T0_322_dig_DB_Bac = pd.concat(frames)

print('# of total Bacteria peptides = ', len(T0_322_dig_DB_Bac))

T0_322_dig_DB_Bac.head()

# of kingdom =  8
# of subkingdom =  1
# of total Bacteria peptides =  9


,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,...,tribe,subtribe,genus,subgenus,species,group,species.1,subgroup,species.2,subspecies
peptide,,,,,,,,,,,,,,,,,,,,,
QGYSNPR,Barnesiella,viscericola,Bacteria,NaN,NaN,NaN,Bacteroidetes,NaN,NaN,Bacteroidia,...,NaN,NaN,NaN,Barnesiella,NaN,NaN,NaN,Barnesiella,viscericola,NaN
DSPPALALK,Verrucomicrobia,bacterium,Bacteria,NaN,NaN,NaN,Verrucomicrobia,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Verrucomicrobia,bacterium,NaN
SVDPYDR,Streptomyces,violaceoruber,Bacteria,NaN,NaN,NaN,Actinobacteria,NaN,NaN,Actinobacteria,...,NaN,NaN,NaN,Streptomyces,NaN,NaN,NaN,Streptomyces,violaceoruber,NaN
NDEALAFLR,Chloroflexi,bacterium,Bacteria,NaN,NaN,NaN,Chloroflexi,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chloroflexi,bacterium,NaN
SGDCTQER,Myxococcaceae,bacterium,Bacteria,NaN,NaN,NaN,Proteobacteria,delta/epsilon,subdivisions,NaN,...,Myxococcaceae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Myxococcaceae,bacterium


### 2. T0 322 trypsin de novo (DN)

In [19]:
# read the data into pandas dataframes
T0_322_dig_DN = pd.read_csv("/home/millieginty/Documents/git-repos/rot-mayer/analyses/unipept/lca/cleaned/T0-322-trypsin-lca-DN.csv", index_col=0)

T0_322_dig_DN.head()

,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,...,tribe,subtribe,genus,subgenus,species,group,species.1,subgroup,species.2,subspecies
peptide,,,,,,,,,,,,,,,,,,,,,
TPEEEER,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAEEEEER,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WAEEEEER,Viridiplantae,Eukaryota,Viridiplantae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
KLPEER,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ESQAWSR,Tardiphaga,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Alphaproteobacteria,NaN,...,NaN,NaN,Tardiphaga,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Take just the bacterial peptides

# keep only the rows where kingdom = Bacteria

T0_322_dig_DN_Bac1 = T0_322_dig_DN.loc[(T0_322_dig_DN['kingdom'] == 'Bacteria')]

T0_322_dig_DN_Bac2 = T0_322_dig_DN.loc[(T0_322_dig_DN['subkingdom'] == 'Bacteria')]

print('# of kingdom = ', len(T0_322_dig_DN_Bac1))

print('# of subkingdom = ', len(T0_322_dig_DN_Bac2))

frames = [T0_322_dig_DN_Bac1, T0_322_dig_DN_Bac2]
  
T0_322_dig_DN_Bac = pd.concat(frames)

print('# of total Bacteria peptides = ', len(T0_322_dig_DN_Bac))

T0_322_dig_DN_Bac.head()

# of kingdom =  100
# of subkingdom =  5
# of total Bacteria peptides =  105


,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,...,tribe,subtribe,genus,subgenus,species,group,species.1,subgroup,species.2,subspecies
peptide,,,,,,,,,,,,,,,,,,,,,
FNSSLAGR,Rhodobacter,viridis,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Alphaproteobacteria,...,NaN,NaN,NaN,Rhodobacter,NaN,NaN,NaN,Rhodobacter,viridis,NaN
WTLEEDAK,Kutzneria,buriramensis,Bacteria,NaN,NaN,NaN,Actinobacteria,NaN,NaN,Actinobacteria,...,NaN,NaN,NaN,Kutzneria,NaN,NaN,NaN,Kutzneria,buriramensis,NaN
VTNAELAEVLAR,Escherichia,coli,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,NaN,NaN,NaN,Escherichia,NaN,NaN,NaN,Escherichia,coli,NaN
QGYSNPR,Barnesiella,viscericola,Bacteria,NaN,NaN,NaN,Bacteroidetes,NaN,NaN,Bacteroidia,...,NaN,NaN,NaN,Barnesiella,NaN,NaN,NaN,Barnesiella,viscericola,NaN
VSLNKVYK,Ehrlichia,minasensis,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Alphaproteobacteria,...,NaN,NaN,NaN,Ehrlichia,NaN,canis,group,NaN,Ehrlichia,minasensis


### 3. T2 323 trypsin DN

In [17]:
# read the data into pandas dataframes
T2_323_dig_DN = pd.read_csv("/home/millieginty/Documents/git-repos/rot-mayer/analyses/unipept/lca/cleaned/T2-323-trypsin-lca-DN.csv", index_col=0)

T2_323_dig_DN.head()

,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,...,tribe,subtribe,genus,subgenus,species,group,species.1,subgroup,species.2,subspecies
peptide,,,,,,,,,,,,,,,,,,,,,
TPEEEER,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HAVSEGTK,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
KQELEDLTK,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TTEEAVR,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CLAYPK,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Take just the bacterial peptides

# keep only the rows where kingdom = Bacteria

T2_323_dig_DN_Bac1 = T2_323_dig_DN.loc[(T2_323_dig_DN['kingdom'] == 'Bacteria')]

T2_323_dig_DN_Bac2 = T2_323_dig_DN.loc[(T2_323_dig_DN['subkingdom'] == 'Bacteria')]

print('# of kingdom = ', len(T2_323_dig_DN_Bac1))

print('# of subkingdom = ', len(T2_323_dig_DN_Bac2))

frames = [T2_323_dig_DN_Bac1, T2_323_dig_DN_Bac2]
  
T2_323_dig_DN_Bac = pd.concat(frames)

print('# of total Bacteria peptides = ', len(T2_323_dig_DN_Bac))

T2_323_dig_DN_Bac.head()

# of kingdom =  125
# of subkingdom =  9
# of total Bacteria peptides =  134


,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,...,tribe,subtribe,genus,subgenus,species,group,species.1,subgroup,species.2,subspecies
peptide,,,,,,,,,,,,,,,,,,,,,
VTNAELAEVLAR,Escherichia,coli,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,NaN,NaN,NaN,Escherichia,NaN,NaN,NaN,Escherichia,coli,NaN
SNGLSEEK,Gammaproteobacteria,bacterium,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gammaproteobacteria,bacterium,NaN
VSLNKVYK,Ehrlichia,minasensis,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Alphaproteobacteria,...,NaN,NaN,NaN,Ehrlichia,NaN,canis,group,NaN,Ehrlichia,minasensis
EVPQELVR,Pseudomonas,cichorii,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,NaN,NaN,NaN,Pseudomonas,NaN,Pseudomonas,syringae,group,NaN,Pseudomonas
LATVLSPR,Gammaproteobacteria,bacterium,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gammaproteobacteria,bacterium,NaN


### 4. T5 324 trypsin DN

In [20]:
# read the data into pandas dataframes
T5_324_dig_DN = pd.read_csv("/home/millieginty/Documents/git-repos/rot-mayer/analyses/unipept/lca/cleaned/T5-324-trypsin-lca-DN.csv", index_col=0)

T5_324_dig_DN.head()

,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,...,tribe,subtribe,genus,subgenus,species,group,species.1,subgroup,species.2,subspecies
peptide,,,,,,,,,,,,,,,,,,,,,
KFDEETK,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TPEEEER,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HLDVDDSGK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SPNETEER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VVEEVVR,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Take just the bacterial peptides

# keep only the rows where kingdom = Bacteria

T5_324_dig_DN_Bac1 = T5_324_dig_DN.loc[(T5_324_dig_DN['kingdom'] == 'Bacteria')]

T5_324_dig_DN_Bac2 = T5_324_dig_DN.loc[(T5_324_dig_DN['subkingdom'] == 'Bacteria')]

print('# of kingdom = ', len(T5_324_dig_DN_Bac1))

print('# of subkingdom = ', len(T5_324_dig_DN_Bac2))

frames = [T5_324_dig_DN_Bac1, T5_324_dig_DN_Bac2]
  
T5_324_dig_DN_Bac = pd.concat(frames)

print('# of total Bacteria peptides = ', len(T5_324_dig_DN_Bac))

T5_324_dig_DN_Bac.head()

# of kingdom =  74
# of subkingdom =  4
# of total Bacteria peptides =  78


,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,...,tribe,subtribe,genus,subgenus,species,group,species.1,subgroup,species.2,subspecies
peptide,,,,,,,,,,,,,,,,,,,,,
VTNAELAEVLAR,Escherichia,coli,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,NaN,NaN,NaN,Escherichia,NaN,NaN,NaN,Escherichia,coli,NaN
QGYSNPR,Barnesiella,viscericola,Bacteria,NaN,NaN,NaN,Bacteroidetes,NaN,NaN,Bacteroidia,...,NaN,NaN,NaN,Barnesiella,NaN,NaN,NaN,Barnesiella,viscericola,NaN
FNSSLAGR,Rhodobacter,viridis,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Alphaproteobacteria,...,NaN,NaN,NaN,Rhodobacter,NaN,NaN,NaN,Rhodobacter,viridis,NaN
WGAPAPR,Halomonas,ilicicola,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,NaN,NaN,NaN,Halomonas,NaN,NaN,NaN,Halomonas,ilicicola,NaN
ANNDVPR,Candidatus,Aureabacteria,Bacteria,NaN,NaN,NaN,Candidatus,Aureabacteria,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 5. T12 325 trypsin DN

In [22]:
# read the data into pandas dataframes
T12_325_dig_DN = pd.read_csv("/home/millieginty/Documents/git-repos/rot-mayer/analyses/unipept/lca/cleaned/T12-325-trypsin-lca-DN.csv", index_col=0)

T12_325_dig_DN.head()

,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,...,tribe,subtribe,genus,subgenus,species,group,species.1,subgroup,species.2,subspecies
peptide,,,,,,,,,,,,,,,,,,,,,
SPPALALK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
KNEAEELK,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FLGNLLTR,Bacteria,Bacteria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DEETKLSK,Biomphalaria,glabrata,Eukaryota,Metazoa,NaN,NaN,Mollusca,NaN,NaN,Gastropoda,...,NaN,NaN,NaN,Biomphalaria,NaN,NaN,NaN,Biomphalaria,glabrata,NaN
HLDVDDSGK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# Take just the bacterial peptides

# keep only the rows where kingdom = Bacteria

T12_325_dig_DN_Bac1 = T12_325_dig_DN.loc[(T12_325_dig_DN['kingdom'] == 'Bacteria')]

T12_325_dig_DN_Bac2 = T12_325_dig_DN.loc[(T12_325_dig_DN['subkingdom'] == 'Bacteria')]

print('# of kingdom = ', len(T12_325_dig_DN_Bac1))

print('# of subkingdom = ', len(T12_325_dig_DN_Bac2))

frames = [T12_325_dig_DN_Bac1, T12_325_dig_DN_Bac2]
  
T12_325_dig_DN_Bac = pd.concat(frames)

print('# of total Bacteria peptides = ', len(T12_325_dig_DN_Bac))

T12_325_dig_DN_Bac.head()

# of kingdom =  135
# of subkingdom =  8
# of total Bacteria peptides =  143


,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,...,tribe,subtribe,genus,subgenus,species,group,species.1,subgroup,species.2,subspecies
peptide,,,,,,,,,,,,,,,,,,,,,
VTNAELAEVLAR,Escherichia,coli,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,NaN,NaN,NaN,Escherichia,NaN,NaN,NaN,Escherichia,coli,NaN
VSNDTAENR,Marinirhabdus,gelatinilytica,Bacteria,NaN,NaN,NaN,Bacteroidetes,NaN,NaN,Flavobacteriia,...,NaN,NaN,NaN,Marinirhabdus,NaN,NaN,NaN,Marinirhabdus,gelatinilytica,NaN
FFLFDDVR,Verrucomicrobia,bacterium,Bacteria,NaN,NaN,NaN,Verrucomicrobia,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Verrucomicrobia,bacterium,NaN
TNFVNPK,Chryseobacterium,arthrosphaerae,Bacteria,NaN,NaN,NaN,Bacteroidetes,NaN,NaN,Flavobacteriia,...,NaN,NaN,NaN,Chryseobacterium,NaN,NaN,NaN,Chryseobacterium,arthrosphaerae,NaN
LATVLSPR,Gammaproteobacteria,bacterium,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gammaproteobacteria,bacterium,NaN


### 6. T12 325 trypsin DB

In [24]:
# read the data into pandas dataframes
T12_325_dig_DB = pd.read_csv("/home/millieginty/Documents/git-repos/rot-mayer/analyses/unipept/lca/cleaned/T12-325-trypsin-lca-DB.csv", index_col=0)

T12_325_dig_DB.head()

,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,...,tribe,subtribe,genus,subgenus,species,group,species.1,subgroup,species.2,subspecies
peptide,,,,,,,,,,,,,,,,,,,,,
LPQVEGTGGDVQPSQDLVR,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VLGQNEAVDAVSNALR,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALDLLDEAASSLR,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DVPGTGNEFVGDFR,Bacillariophyta,Eukaryota,NaN,NaN,NaN,Bacillariophyta,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VTDAELAEVLAR,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# Take just the bacterial peptides

# keep only the rows where kingdom = Bacteria

T12_325_dig_DB_Bac1 = T12_325_dig_DB.loc[(T12_325_dig_DB['kingdom'] == 'Bacteria')]

T12_325_dig_DB_Bac2 = T12_325_dig_DB.loc[(T12_325_dig_DB['subkingdom'] == 'Bacteria')]

print('# of kingdom = ', len(T12_325_dig_DB_Bac1))

print('# of subkingdom = ', len(T12_325_dig_DB_Bac2))

frames = [T12_325_dig_DB_Bac1, T12_325_dig_DB_Bac2]
  
T12_325_dig_DB_Bac = pd.concat(frames)

print('# of total Bacteria peptides = ', len(T12_325_dig_DB_Bac))

T12_325_dig_DB_Bac.head()

# of kingdom =  1
# of subkingdom =  1
# of total Bacteria peptides =  2


,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,...,tribe,subtribe,genus,subgenus,species,group,species.1,subgroup,species.2,subspecies
peptide,,,,,,,,,,,,,,,,,,,,,
NDEALAFLR,Chloroflexi,bacterium,Bacteria,NaN,NaN,NaN,Chloroflexi,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chloroflexi,bacterium,NaN
DKFDEETK,Candidatus,Cloacimonetes,bacterium,Bacteria,NaN,NaN,NaN,Candidatus,Cloacimonetes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Candidatus


### Combining all bacterial peptide results

In [28]:
frames = [T0_322_dig_DB_Bac, T0_322_dig_DN_Bac, T2_323_dig_DN_Bac, \
          T5_324_dig_DN_Bac, T12_325_dig_DN_Bac, T12_325_dig_DB_Bac]
  
combo_Bac = pd.concat(frames)

combo_Bac_dd = pd.DataFrame.drop_duplicates(combo_Bac)

print('# of total Bacteria peptides = ', len(combo_Bac))
print('# of nonredudant Bacteria peptides = ', len(combo_Bac_dd))

combo_Bac.head()

# of total Bacteria peptides =  471
# of nonredudant Bacteria peptides =  342


,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,...,tribe,subtribe,genus,subgenus,species,group,species.1,subgroup,species.2,subspecies
peptide,,,,,,,,,,,,,,,,,,,,,
QGYSNPR,Barnesiella,viscericola,Bacteria,NaN,NaN,NaN,Bacteroidetes,NaN,NaN,Bacteroidia,...,NaN,NaN,NaN,Barnesiella,NaN,NaN,NaN,Barnesiella,viscericola,NaN
DSPPALALK,Verrucomicrobia,bacterium,Bacteria,NaN,NaN,NaN,Verrucomicrobia,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Verrucomicrobia,bacterium,NaN
SVDPYDR,Streptomyces,violaceoruber,Bacteria,NaN,NaN,NaN,Actinobacteria,NaN,NaN,Actinobacteria,...,NaN,NaN,NaN,Streptomyces,NaN,NaN,NaN,Streptomyces,violaceoruber,NaN
NDEALAFLR,Chloroflexi,bacterium,Bacteria,NaN,NaN,NaN,Chloroflexi,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chloroflexi,bacterium,NaN
SGDCTQER,Myxococcaceae,bacterium,Bacteria,NaN,NaN,NaN,Proteobacteria,delta/epsilon,subdivisions,NaN,...,Myxococcaceae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Myxococcaceae,bacterium


In [35]:
# keep only peptide column

combo_Bac_dd.reset_index(inplace=True)

bac_pep = combo_Bac_dd[["peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
bac_pep.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/analyses/unipept/GO-terms/nonredundant_Bac_peptides.txt", header=False, index=False)

bac_pep.head()

ValueError: cannot insert level_0, already exists

### 251 peptides (73%) have at least one GO-term assigned to them after running nonredudant peptides through UniPept

!()